In [16]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.linear_model import LogisticRegression
import seaborn as sns

In [2]:
SIZE = 224
input_shape = (224, 224, 3)
EPOCHS = 100
BS = 64
output_ = 2

In [3]:
train_data_names = []
train_data = []
train_labels = []

In [4]:
for data in glob.glob('./Dataset/train/*.png'):
    train_data_names.append(data)
    img = cv2.imread(data)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (SIZE, SIZE))
    train_data.append(img)
    if data.split('/')[-1].endswith('g.png'):
        train_labels.append(1)
    else:
        train_labels.append(0)

In [5]:
train_data = np.array(train_data) / 255.0
train_labels = np.array(train_labels)

In [6]:
train_labels = to_categorical(train_labels, num_classes=output_)

In [7]:
train_data, val_data, train_labels, test_labels = train_test_split(
    train_data, train_labels, test_size=0.2, random_state=42
)

In [8]:
base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = False

data_augmentation = Sequential([
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomFlip("horizontal_and_vertical"),
], name="data_augmentation")

inputs = layers.Input(shape=input_shape)
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(output_, activation='softmax')(x)
model = Model(inputs, outputs)

model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy']
)

model.summary()

earlyStopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True  
)

history = model.fit(
    train_data, train_labels,
    batch_size=BS,
    epochs=EPOCHS,
    validation_data=(val_data, test_labels),
    callbacks=[earlyStopping]
)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ data_augmentation (Sequential)       │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_v3 (Functional)            │ (None, 5, 5, 2048)          │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 51200)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      13,107,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             514 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,910,754 (133.17 MB)

 Trainable params: 13,107,970 (50.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.5202 - loss: 2.8639 - val_accuracy: 0.4176 - val_loss: 1.1661
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 885ms/step - accuracy: 0.5325 - loss: 0.8499 - val_accuracy: 0.6044 - val_loss: 0.6511
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 816ms/step - accuracy: 0.5694 - loss: 0.6952 - val_accuracy: 0.5934 - val_loss: 0.6713
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 863ms/step - accuracy: 0.5736 - loss: 0.6730 - val_accuracy: 0.6044 - val_loss: 0.6604
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 813ms/step - accuracy: 0.6075 - loss: 0.6489 - val_accuracy: 0.6264 - val_loss: 0.6181
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 783ms/step - accuracy: 0.6518 - loss: 0.6338 - val_accuracy: 0.6044 - val_loss: 0.6104
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 855ms/step - accuracy: 0.6638 - loss: 0.6066 - val_accuracy: 0.7253 - val_loss: 0.5769
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 924ms/step - accuracy: 0.7030 - loss: 0.5664 - val_accuracy: 0.6813 - val

In [9]:
intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-2].output) 
intermediate_output_train = intermediate_layer_model.predict(train_data)
intermediate_output_val = intermediate_layer_model.predict(val_data)

12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 467ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 320ms/step


In [10]:
train_labels = np.argmax(train_labels, axis=1)
test_labels = np.argmax(test_labels, axis=1)

In [11]:
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(intermediate_output_train, train_labels)

LogisticRegression(max_iter=1000)

In [12]:
y_pred = logreg_model.predict(intermediate_output_val)

In [13]:
accuracy = accuracy_score(test_labels, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(test_labels, y_pred, average='macro')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

Accuracy: 0.8241758241758241
Precision: 0.8173076923076923, Recall: 0.8220720720720721, F1 Score: 0.8192651439920556


In [19]:
conf_matrix = confusion_matrix(test_labels, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

conf_matrix_array = conf_matrix.tolist()
print("Confusion Matrix as Array:")
print(conf_matrix_array)

Confusion Matrix:
[[45  9]
 [ 7 30]]
Confusion Matrix as Array:
[[45, 9], [7, 30]]
